In [1]:
from populations.bbh_models import get_models


PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


In [ ]:
get_models('../OneChannel_Flows/models_reduced.hdf5',['CE'],['mchirp'])

In [2]:
import pandas as pd
import numpy as np
import h5py

from scipy.special import logit
from scipy.special import expit

def logistic(data,rescaling=False, wholedataset=True, max =1, rescale_max=1):
    if rescaling:
        if wholedataset:
            rescale_max = np.max(data) + 0.01
        else:
            rescale_max = rescale_max
        data /= rescale_max
    else:
        rescale_max = None
    data = logit(data)
    if wholedataset:
        max = np.max(data)
    else:
        max = max
    data /= max
    return([data, max, rescale_max])

def expistic(data, max, rescale_max=None):
    data*=max
    data = expit(data)
    if rescale_max != None:
        data *=rescale_max
    return(data)

def get_model_keys(path):
    alpha_val = '10'
    all_models = []
    models = []
    def find_submodels(name, obj):
        if isinstance(obj, h5py.Dataset):
            all_models.append(name.rsplit('/', 1)[0])
            
    f = h5py.File(path, 'r')
    f.visititems(find_submodels)
    # get all unique models
    all_models = sorted(list(set(all_models)))
    f.close()

    # use only models with given alpha value
    for model in all_models:
        if 'alpha' in model:
            if 'alpha'+alpha_val in model:
                models.append('/'+model)
        else:
            models.append('/' + model)
    return(np.split(np.array(models), 5))

def get_model_keys_CE(path):
    all_models = []
    models = []
    def find_submodels(name, obj):
        if isinstance(obj, h5py.Dataset):
            all_models.append(name.rsplit('/', 1)[0])
            
    f = h5py.File(path, 'r')
    f.visititems(find_submodels)
    # get all unique models
    all_models = sorted(list(set(all_models)))
    f.close()

    # use only models with given alpha value
    for model in all_models:
        if 'CE' in model:
            models.append('/'+model)
    return(np.split(np.array(models), 4))

def read_hdf5(path, all_alpha=False):
    if all_alpha:
        popsynth_outputs = {}
        models = np.asarray(get_model_keys_CE(path))
        for i in range(models.shape[0]):
            for j in range(models.shape[1]):
                popsynth_outputs[i,j]=pd.read_hdf(path, key=models[i,j])
    else:
        popsynth_outputs = {}
        models = np.asarray(get_model_keys(path))
        for i in range(models.shape[0]):
            for j in range(models.shape[1]):
                popsynth_outputs[i,j]=pd.read_hdf(path, key=models[i,j])

    return(popsynth_outputs)

def plot_histogram(model_id, param, axes, models_path):
    popsynth_outputs = read_hdf5(models_path)
    axes.hist(popsynth_outputs[model_id][param])

In [1]:
models_path ='/Users/stormcolloms/Documents/PhD/Project_work/OneChannel_Flows/models_reduced.hdf5'
samples = read_hdf5(models_path, all_alpha=False)

cond_inputs = 1

channel_label = 'SMT'
params = ['mchirp','q', 'chieff', 'z']
no_params = 4

chi_b = [0.,0.1,0.2,0.5]
alpha = [0.2,0.5,1.,2.,5.]


NameError: name 'read_hdf5' is not defined

In [1]:
import populations.bbh_models as read_models
from populations.utils.flow import NFlow

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


glasflow is using its own internal version of nflows


In [2]:
channels = ['CE']
file_path='/Users/stormcolloms/Documents/PhD/Project_work/OneChannel_Flows/models_reduced.hdf5'
params =['mchirp', 'q','chieff', 'z']

flow = read_models.get_models(file_path, channels, params)

In [3]:
flow_model= flow['GC']
print(flow)

In [5]:
flow.load_model('test_training.pt')